In [1]:
from Bio import SeqIO

In [1]:
def overlaps(cnv_len, file_len, taken_places):
    start = random.randint(1, file_len - cnv_len)
    stop = start + cnv_len
    for start_taken, stop_taken in taken_places:
        if (start >= start_taken and start <= stop_taken) or \
            (stop <= stop_taken and stop >= start_taken):
            return False
    return True


NameError: name 'SeqIO' is not defined

In [21]:
import random
CNV_num = 20
with open("Sus_scrofa.Sscrofa11.1.dna.chromosome.1.fa") as handle:
    taken_places = []
    for record in SeqIO.parse(handle, "fasta"):
        record_str = str(record.seq)
        file_len = len(record.seq)
        for cnv in range(CNV_num):
            cnv_range = random.randint(50,100000)
            cnv_type = random.choice(['del','dup'])
            if cnv_type == 'dup':
                print("Creating duplication")
                print(taken_places)
                wrong_cnv_coords = True
                while wrong_cnv_coords: 
                    start = random.randint(1, file_len - cnv_range)
                    stop = start + cnv_range + 1
                    if taken_places:
                        for start_taken, stop_taken in taken_places:
                            if not (start >= start_taken and start <= stop_taken) or not \
                                (stop <= stop_taken and stop >= start_taken):
                                wrong_cnv_coords = False
                    else:
                        wrong_cnv_coords = False
                record_str = record_str[:start] + random.choice(["A","C","G","T"]) * cnv_range + record_str[stop:]    
                taken_places.append((start, stop))
            elif cnv_type == 'del':
                print("Creating deletion")
                print(taken_places)
                wrong_cnv_coords = True
                while wrong_cnv_coords: 
                    start = random.randint(1, file_len - cnv_range)
                    stop = start + cnv_range + 1
                    if taken_places:
                        for start_taken, stop_taken in taken_places:
                            if not (start >= start_taken and start <= stop_taken) or not \
                                (stop <= stop_taken and stop >= start_taken):
                                wrong_cnv_coords = False
                    else:
                        wrong_cnv_coords = False
                record_str = record_str[:start] + record_str[stop:]    
                taken_places.append((start, stop))
                
    print(taken_places)
                        
 





Creating duplication
[]
Creating duplication
[(34289351, 34345301)]
Creating duplication
[(34289351, 34345301), (215701130, 215799633)]
Creating duplication
[(34289351, 34345301), (215701130, 215799633), (49898482, 49935704)]
Creating deletion
[(34289351, 34345301), (215701130, 215799633), (49898482, 49935704), (187890662, 187935508)]
Creating deletion
[(34289351, 34345301), (215701130, 215799633), (49898482, 49935704), (187890662, 187935508), (200517868, 200569990)]
Creating duplication
[(34289351, 34345301), (215701130, 215799633), (49898482, 49935704), (187890662, 187935508), (200517868, 200569990), (108079867, 108176335)]
Creating deletion
[(34289351, 34345301), (215701130, 215799633), (49898482, 49935704), (187890662, 187935508), (200517868, 200569990), (108079867, 108176335), (38490156, 38580980)]
Creating duplication
[(34289351, 34345301), (215701130, 215799633), (49898482, 49935704), (187890662, 187935508), (200517868, 200569990), (108079867, 108176335), (38490156, 38580980), (

In [16]:
with open("Sus_scrofa.Sscrofa11.1.dna.chromosome.1.fa") as handle:
    taken_places = []
    for record in SeqIO.parse(handle, "fasta"):
        record_str = str(record.seq)
        print(len(record_str))
        record_str = record_str[:1] + ("T" * 9) + record_str[10:]
        print(len(record_str))
        print(record_str[1:20])

274330532
274330532
TTTTTTTTTTGTCATTTCT


In [15]:
"e" * 6

'eeeeee'

In [7]:
import pandas as pd
import numpy as np
from Bio import SeqIO
import random
import os
df_del = pd.DataFrame(columns = [1,2,3,4])
df_dup = pd.DataFrame(columns = [1,2,3,4])
pathOut = "."
if pathOut != "" and pathOut.endswith("/") == False:
    pathOut += "/"
out = open(pathOut + "chrs.bed","w")
print("Generating duplication and deletion coordinates")
for r in SeqIO.parse(open("Sus_scrofa.Sscrofa11.1.dna.chromosome.1.fa"),"fasta"):
    out.write("\t".join([r.id,"1",str(len(str(r.seq)))]) + "\n")
    dup_lengths = []
    del_lengths = []
    cnv_count = round((len(str(r.seq))/1000000)*50)
    while len(dup_lengths) < cnv_count:
        x = round(np.random.normal(1000, 50, 1)[0])
        if x > 50:
            dup_lengths.append(x)
    while len(del_lengths) < cnv_count:
        x = round(np.random.normal(1000, 50, 1)[0])
        if x > 50:
            del_lengths.append(x)
    dup_start = list(np.random.randint(len(str(r.seq)), size=(1, cnv_count))[0])
    del_start = list(np.random.randint(len(str(r.seq)), size=(1, cnv_count))[0])
    dup_ends = list(map(int,[a + b for a, b in zip(dup_start, dup_lengths)]))
    del_ends = list(map(int,[a + b for a, b in zip(del_start, del_lengths)]))
    dups = pd.DataFrame({1:[r.id]*cnv_count,2:dup_start,3:dup_ends,4:dup_lengths})
    dels = pd.DataFrame({1:[r.id]*cnv_count,2:del_start,3:del_ends,4:del_lengths})
    df_dup = df_dup.append(dups)
    df_del = df_del.append(dels)
out.close()
print(df_dup)
df_dup.to_csv(pathOut + "dup.bed",header=False,index=False,sep="\t")
df_del.to_csv(pathOut + "del.bed",header=False,index=False,sep="\t")
os.system(bedtools_path + " sort -i " + pathOut + "dup.bed | " + bedtools_path + " merge -i stdin > " + pathOut + "dup2.bed")
os.system(bedtools_path + " sort -i " + pathOut + "del.bed | " + bedtools_path + " merge -i stdin > " + pathOut + "del2.bed")
if args.TE is not None:
    os.system(bedtools_path + " intersect -v -wa -a "+ pathOut + "del2.bed -b " + args.TE + " -f " + str(args.CUTOFF) + " > "+ pathOut + "del3.bed")
    os.system(bedtools_path + " intersect -v -wa -a "+ pathOut + "dup2.bed -b " + args.TE + " -f " + str(args.CUTOFF) + " > "+ pathOut + "dup3.bed")
elif args.TE is None:
    os.system("cp "+ pathOut + "del2.bed "+ pathOut + "del3.bed")
    os.system("cp "+ pathOut + "dup2.bed "+ pathOut + "dup3.bed")
os.system(bedtools_path + " intersect -wa -v -a " + pathOut + "dup3.bed -b " + pathOut + "del3.bed > " + pathOut + "dup4.bed")
os.system(bedtools_path + " intersect -wa -v -a " + pathOut + "del3.bed -b " + pathOut + "dup3.bed > " + pathOut + "del4.bed")
no_chrs = list(range(1, int(args.NUMBER)+1))
chr_freq = {}
for i in no_chrs:
    chr_freq[i] = i/args.NUMBER
no_chrs = list(range(1, int(args.NUMBER)+1))
chr_freq = {}
if args.QUIET == False:
    print("Generating duplication and deletion frequencies")
for i in no_chrs:
    chr_freq[i] = round(i/args.NUMBER,3)
for i in ["del","dup"]:
    out = open(pathOut + str(i) + "5.bed","w")
    for line in open(pathOut + i + "4.bed"):
        if i == "del":
            num = random.randint(1,args.NUMBER)
            out.write(line.rstrip() + "\tdel\t" + str(chr_freq[num]) + "\t0\n")
        elif i == "dup":
            num = random.randint(1,args.NUMBER)
            count = np.random.choice([2,3,4,5,6,7,8,9,10], 1, p=[0.5, 0.1, 0.1, 0.05, 0.05,0.05,0.05,0.05,0.05])[0]
            freqs = num/args.NUMBER
            cp = (count*freqs) + ((1-freqs) * 1)
            while cp == 1.0:
                num = random.randint(1,args.NUMBER)
                count = np.random.choice([2,3,4,5,6,7,8,9,10], 1, p=[0.5, 0.1, 0.1, 0.05, 0.05,0.05,0.05,0.05,0.05])[0]
            out.write(line.rstrip() + "\tdup\t" + str(chr_freq[num]) + "\t" + str(count) + "\n")
    out.close()
    for j in chr_freq:
        out = open(pathOut + i + "." + str(j) + ".bed","w")
        for line in open(pathOut + i + "5.bed"):
            if float(line.split()[4]) >= chr_freq[j]:
                out.write(line)
        out.close()
if args.QUIET == False:
    print("Removing overlaps, generating total file")
for i in no_chrs:
    print("Creating bedfiles for sample " + str(i))
    os.system("bedtools makewindows -b " + pathOut + "chrs.bed -w 5 > " + pathOut + "normal." + str(i) + ".bed")
    os.system(bedtools_path + " intersect -v -wa -a " + pathOut + "normal." + str(i) + ".bed -b " + pathOut + "dup." + str(i) + ".bed | " + bedtools_path + " intersect -v -wa -a stdin -b " + pathOut + "del." + str(i) + ".bed | " + bedtools_path + " sort -i stdin | " + bedtools_path + " merge -i stdin > " + pathOut + "normal2." + str(i) + ".bed")
    out = open(pathOut + "normal3." + str(i) + ".bed","w")
    for line in open(pathOut + "normal2." + str(i) + ".bed"):
        out.write(line.rstrip() + "\tnormal\t1\t1\n")
    out.close()
    os.system("cat " + pathOut + "normal3." + str(i) + ".bed " + pathOut + "dup." + str(i) + ".bed " + pathOut + "del." + str(i) + ".bed | " + bedtools_path + " sort -i stdin > " + pathOut + "total." + str(i) + ".bed")
    os.remove(pathOut + "normal3." + str(i) + ".bed")
    os.remove(pathOut + "normal2." + str(i) + ".bed")
    os.remove(pathOut + "normal." + str(i) + ".bed")
os.remove(pathOut + "del.bed")
os.remove(pathOut + "del2.bed")
os.remove(pathOut + "del3.bed")
os.remove(pathOut + "del4.bed")
os.remove(pathOut + "del5.bed")
os.remove(pathOut + "dup.bed")
os.remove(pathOut + "dup2.bed")
os.remove(pathOut + "dup3.bed")
os.remove(pathOut + "dup4.bed")
os.remove(pathOut + "dup5.bed")
os.remove(pathOut + "chrs.bed") 

Generating duplication and deletion coordinates
       1          2          3     4
0      1  225001747  225002689   942
1      1   89402865   89403871  1006
2      1   54498095   54499074   979
3      1  214044384  214045421  1037
4      1  168533841  168534872  1031
...   ..        ...        ...   ...
13712  1  220135076  220136194  1118
13713  1  201601797  201602775   978
13714  1   99490232   99491251  1019
13715  1   54353951   54354928   977
13716  1  153210590  153211588   998

[13717 rows x 4 columns]


/tmp/ipykernel_4171/2451843187.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_dup = df_dup.append(dups)
/tmp/ipykernel_4171/2451843187.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_del = df_del.append(dels)
